In [1]:
import collections
import pandas as pd
import pyreadr
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from venn import venn
import numpy as np
import math
import scipy

/var/folders/y8/66w8_8kj0dx2znq80cdbl_rw0000gn/T/ipykernel_24150/2185171821.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
clinical_df = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/figurecode/data/20221109_TRACERx421_all_patient_df.rds')
clinical_df = pd.DataFrame(clinical_df[None])
clinical_df.head()

,cruk_id,tumour_id_muttable_cruk,tumour_id_per_patient,age,sex,ethnicity,cigs_perday,years_smoking,packyears,smoking_status_merged,...,os_time,cens_dfs,dfs_time,cens_dfs_any_event,dfs_time_any_event,cens_lung_event,lung_event_time,Relapse_cat,Relapse_cat_new,tx100
0,CRUK0034,CRUK0034,CRUK0034,68,Female,White- Irish,20.0,35,35.000,Ex-Smoker,...,1849,0,1849,0,1849,0,1849,NaN,No rec,True
1,CRUK0150,CRUK0150,CRUK0150,81,Male,White- British,44.5,49,109.025,Ex-Smoker,...,1362,1,1362,1,1362,0,1362,NaN,No rec,False
2,CRUK0159,CRUK0159,CRUK0159,60,Male,White- British,20.0,38,38.000,Smoker,...,2224,1,2224,1,1935,0,2224,Second primary lung,No rec,False
3,CRUK0090,CRUK0090,CRUK0090,65,Male,White- British,10.0,35,17.500,Ex-Smoker,...,2365,1,1143,1,1143,1,1143,Intrathoracic,Intrathoracic,True
4,CRUK0045,CRUK0045,CRUK0045,85,Male,White- British,10.0,25,12.500,Ex-Smoker,...,986,1,917,1,917,1,917,Intrathoracic,Intrathoracic,True


In [4]:
tumor_df = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/LCCE_converted_data/20221109_TRACERx421_all_tumour_df.rds')
tumor_df = pd.DataFrame(tumor_df[None])
tumor_df.head()

,tumour_id,patient_id,clinical_sex,age,ethnicity,Histology_per_tumour_id_muttable,histology_3,LUAD_pred_subtype_with.IMA_per_tumour,site_per_lesion,pT_stage_per_lesion,pN_stage_per_lesion,size_pathology_per_lesion,vascular_invasion_per_lesion,pleural_invasion_per_lesion,smoking_status_merged,cigs_perday,years_smoking,pack_years,is.family.lung,pathologyTNM
0,CRUK0034,CRUK0034,Female,68,White- Irish,Invasive adenocarcinoma,LUAD,acinar,Right Upper Lobe,2a,0,40,Yes,No,Ex-Smoker,20.0,35,35.000,True,IB
1,CRUK0150,CRUK0150,Male,81,White- British,LCNEC,Other,NaN,Left Upper Lobe,3,0,75,Yes,No,Ex-Smoker,44.5,49,109.025,False,IIB
2,CRUK0159,CRUK0159,Male,60,White- British,Invasive adenocarcinoma,LUAD,acinar,Right Upper Lobe,2a,0,36,Yes,Yes,Smoker,20.0,38,38.000,False,IB
3,CRUK0090,CRUK0090,Male,65,White- British,Squamous cell carcinoma,LUSC,NaN,Right Upper Lobe,2a,1,35,Yes,Yes,Ex-Smoker,10.0,35,17.500,False,IIA
4,CRUK0045,CRUK0045,Male,85,White- British,Invasive adenocarcinoma,LUAD,papillary,Left Lower Lobe,2a,0,48,Yes,Yes,Ex-Smoker,10.0,25,12.500,False,IB


In [5]:
mut_table = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/LCCE_converted_data/20221109_TRACERx421_mutation_table.rds')
mut_table = pd.DataFrame(mut_table[None])
mut_table.head()

,patient_id,tumour_id,mutation_id,chr,start,stop,ref,var,Hugo_Symbol,func,...,AAChange,GL_VAF,GL_nAlt,GL_depth,ITHState,PyCloneClonal_SC,PyCloneCluster_SC,cleanCluster_SC,combTiming_SC,DriverMut
0,CRUK0005,CRUK0005,CRUK0005:19:34291428:G,chr19,34291428,34291428,G,T,KCTD15,intronic,...,NaN,0,0,187,1,S,3.0,True,subclonal,False
1,CRUK0005,CRUK0005,CRUK0005:19:9084770:C,chr19,9084770,9084770,C,A,MUC16,exonic,...,p.V2349L,0.39,2,510,1,C,2.0,True,early,False
2,CRUK0005,CRUK0005,CRUK0005:1:2160418:G,chr1,2160418,2160418,G,T,SKI,exonic,...,p.P71P,0,0,114,1,C,2.0,True,early,False
3,CRUK0005,CRUK0005,CRUK0005:19:40095981:G,chr19,40095981,40095981,G,T,LGALS13,exonic,...,p.G86C,0,0,659,1,C,2.0,True,early,False
4,CRUK0005,CRUK0005,CRUK0005:19:41313703:C,chr19,41313703,41313703,C,G,RAB4B-EGLN2,ncRNA_intronic,...,NaN,0,0,894,3,S,5.0,True,subclonal,False


In [6]:
TP53_present = mut_table['Hugo_Symbol']=='TP53'
TP53_driver = mut_table[(mut_table['Hugo_Symbol'] == 'TP53') & (mut_table['DriverMut'] == True)]
TP53_driver

,patient_id,tumour_id,mutation_id,chr,start,stop,ref,var,Hugo_Symbol,func,...,AAChange,GL_VAF,GL_nAlt,GL_depth,ITHState,PyCloneClonal_SC,PyCloneCluster_SC,cleanCluster_SC,combTiming_SC,DriverMut
113,CRUK0005,CRUK0005,CRUK0005:17:7578194:-,chr17,7578194,7578194,-,CAC,TP53,exonic,...,p.P87delinsVP,0,0,356,1,C,2.0,True,early,True
4295,CRUK0196,CRUK0196,CRUK0196:17:7578461:C,chr17,7578461,7578461,C,A,TP53,exonic,...,p.V25F,0,0,657,1,C,1.0,True,early,True
5383,CRUK0023,CRUK0023,CRUK0023:17:7577538:C,chr17,7577538,7577538,C,T,TP53,exonic,...,p.R116Q,0,0,211,3,S,25.0,False,subclonal,True
7583,CRUK0014,CRUK0014,CRUK0014:17:7578205:C,chr17,7578205,7578205,C,A,TP53,exonic,...,p.S83I,0,0,481,1,C,1.0,True,Unknown,True
8026,CRUK0223,CRUK0223_Tumour1,CRUK0223_Tumour1:17:7577046:C,chr17,7577046,7577046,C,A,TP53,exonic,...,p.E166X,0,0,984,1,C,1.0,True,early,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375562,CRUK0511,CRUK0511,CRUK0511:17:7577551:C,chr17,7577551,7577551,C,A,TP53,exonic,...,p.G112C,0,0,263,1,C,1.0,True,early,True
376627,CRUK0666,CRUK0666,CRUK0666:17:7578526:C,chr17,7578526,7578526,C,T,TP53,exonic,...,p.C3Y,0,0,297,1,C,1.0,True,Unknown,True
377321,CRUK0678,CRUK0678,CRUK0678:17:7577559:G,chr17,7577559,7577559,G,C,TP53,exonic,...,p.S109C,0,0,264,1,C,1.0,True,early,True
378314,CRUK0787,CRUK0787,CRUK0787:17:7577120:C,chr17,7577120,7577120,C,T,TP53,exonic,...,p.R141H,0,0,375,1,C,1.0,True,early,True


In [64]:
four_genes_driver = []
for index, row in clinical_df.iterrows():
    four_genes_driver.append([row['cruk_id'],0,0,0,0])
four_genes_driver

[['CRUK0001', 0, 0, 0, 0],
 ['CRUK0002', 0, 0, 0, 0],
 ['CRUK0003', 0, 0, 0, 0],
 ['CRUK0004', 0, 0, 0, 0],
 ['CRUK0005', 0, 0, 0, 0],
 ['CRUK0006', 0, 0, 0, 0],
 ['CRUK0007', 0, 0, 0, 0],
 ['CRUK0008', 0, 0, 0, 0],
 ['CRUK0009', 0, 0, 0, 0],
 ['CRUK0010', 0, 0, 0, 0],
 ['CRUK0011', 0, 0, 0, 0],
 ['CRUK0012', 0, 0, 0, 0],
 ['CRUK0013', 0, 0, 0, 0],
 ['CRUK0014', 0, 0, 0, 0],
 ['CRUK0015', 0, 0, 0, 0],
 ['CRUK0016', 0, 0, 0, 0],
 ['CRUK0017', 0, 0, 0, 0],
 ['CRUK0018', 0, 0, 0, 0],
 ['CRUK0019', 0, 0, 0, 0],
 ['CRUK0020', 0, 0, 0, 0],
 ['CRUK0021', 0, 0, 0, 0],
 ['CRUK0022', 0, 0, 0, 0],
 ['CRUK0023', 0, 0, 0, 0],
 ['CRUK0024', 0, 0, 0, 0],
 ['CRUK0025', 0, 0, 0, 0],
 ['CRUK0026', 0, 0, 0, 0],
 ['CRUK0027', 0, 0, 0, 0],
 ['CRUK0028', 0, 0, 0, 0],
 ['CRUK0029', 0, 0, 0, 0],
 ['CRUK0030', 0, 0, 0, 0],
 ['CRUK0032', 0, 0, 0, 0],
 ['CRUK0033', 0, 0, 0, 0],
 ['CRUK0034', 0, 0, 0, 0],
 ['CRUK0035', 0, 0, 0, 0],
 ['CRUK0036', 0, 0, 0, 0],
 ['CRUK0037', 0, 0, 0, 0],
 ['CRUK0038', 0, 0, 0, 0],
 

In [41]:
TP53_driver

,patient_id,tumour_id,mutation_id,chr,start,stop,ref,var,Hugo_Symbol,func,...,AAChange,GL_VAF,GL_nAlt,GL_depth,ITHState,PyCloneClonal_SC,PyCloneCluster_SC,cleanCluster_SC,combTiming_SC,DriverMut
113,CRUK0005,CRUK0005,CRUK0005:17:7578194:-,chr17,7578194,7578194,-,CAC,TP53,exonic,...,p.P87delinsVP,0,0,356,1,C,2.0,True,early,True
4295,CRUK0196,CRUK0196,CRUK0196:17:7578461:C,chr17,7578461,7578461,C,A,TP53,exonic,...,p.V25F,0,0,657,1,C,1.0,True,early,True
5383,CRUK0023,CRUK0023,CRUK0023:17:7577538:C,chr17,7577538,7577538,C,T,TP53,exonic,...,p.R116Q,0,0,211,3,S,25.0,False,subclonal,True
7583,CRUK0014,CRUK0014,CRUK0014:17:7578205:C,chr17,7578205,7578205,C,A,TP53,exonic,...,p.S83I,0,0,481,1,C,1.0,True,Unknown,True
8026,CRUK0223,CRUK0223_Tumour1,CRUK0223_Tumour1:17:7577046:C,chr17,7577046,7577046,C,A,TP53,exonic,...,p.E166X,0,0,984,1,C,1.0,True,early,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375562,CRUK0511,CRUK0511,CRUK0511:17:7577551:C,chr17,7577551,7577551,C,A,TP53,exonic,...,p.G112C,0,0,263,1,C,1.0,True,early,True
376627,CRUK0666,CRUK0666,CRUK0666:17:7578526:C,chr17,7578526,7578526,C,T,TP53,exonic,...,p.C3Y,0,0,297,1,C,1.0,True,Unknown,True
377321,CRUK0678,CRUK0678,CRUK0678:17:7577559:G,chr17,7577559,7577559,G,C,TP53,exonic,...,p.S109C,0,0,264,1,C,1.0,True,early,True
378314,CRUK0787,CRUK0787,CRUK0787:17:7577120:C,chr17,7577120,7577120,C,T,TP53,exonic,...,p.R141H,0,0,375,1,C,1.0,True,early,True


In [65]:
clinical_df['TP53_present']=False
for index1, row1 in mut_table[TP53_present].iterrows():
    for index2, row2 in clinical_df.iterrows():
        if row1['patient_id'] == row2['cruk_id']:
            clinical_df.at[index2, 'TP53_present'] = True

for index1, row1 in TP53_driver.iterrows():
    for index2, row2 in clinical_df.iterrows():
        if row1['patient_id'] == row2['cruk_id']:
            for i in four_genes_driver:
                if i[0] == row1['patient_id']:
                    i[1] = 1
four_genes_driver

[['CRUK0001', 1, 0, 0, 0],
 ['CRUK0002', 0, 0, 0, 0],
 ['CRUK0003', 0, 0, 0, 0],
 ['CRUK0004', 1, 0, 0, 0],
 ['CRUK0005', 1, 0, 0, 0],
 ['CRUK0006', 1, 0, 0, 0],
 ['CRUK0007', 0, 0, 0, 0],
 ['CRUK0008', 0, 0, 0, 0],
 ['CRUK0009', 1, 0, 0, 0],
 ['CRUK0010', 0, 0, 0, 0],
 ['CRUK0011', 0, 0, 0, 0],
 ['CRUK0012', 0, 0, 0, 0],
 ['CRUK0013', 0, 0, 0, 0],
 ['CRUK0014', 1, 0, 0, 0],
 ['CRUK0015', 1, 0, 0, 0],
 ['CRUK0016', 1, 0, 0, 0],
 ['CRUK0017', 1, 0, 0, 0],
 ['CRUK0018', 0, 0, 0, 0],
 ['CRUK0019', 0, 0, 0, 0],
 ['CRUK0020', 1, 0, 0, 0],
 ['CRUK0021', 1, 0, 0, 0],
 ['CRUK0022', 1, 0, 0, 0],
 ['CRUK0023', 1, 0, 0, 0],
 ['CRUK0024', 1, 0, 0, 0],
 ['CRUK0025', 1, 0, 0, 0],
 ['CRUK0026', 1, 0, 0, 0],
 ['CRUK0027', 0, 0, 0, 0],
 ['CRUK0028', 1, 0, 0, 0],
 ['CRUK0029', 1, 0, 0, 0],
 ['CRUK0030', 1, 0, 0, 0],
 ['CRUK0032', 0, 0, 0, 0],
 ['CRUK0033', 0, 0, 0, 0],
 ['CRUK0034', 0, 0, 0, 0],
 ['CRUK0035', 1, 0, 0, 0],
 ['CRUK0036', 0, 0, 0, 0],
 ['CRUK0037', 0, 0, 0, 0],
 ['CRUK0038', 0, 0, 0, 0],
 

In [12]:
EGFR_present = mut_table['Hugo_Symbol']=='EGFR'
EGFR_driver = mut_table[(mut_table['Hugo_Symbol'] == 'EGFR') & (mut_table['DriverMut'] == True)]
EGFR_driver

,patient_id,tumour_id,mutation_id,chr,start,stop,ref,var,Hugo_Symbol,func,...,AAChange,GL_VAF,GL_nAlt,GL_depth,ITHState,PyCloneClonal_SC,PyCloneCluster_SC,cleanCluster_SC,combTiming_SC,DriverMut
6444,CRUK0010,CRUK0010,CRUK0010:7:55242466:G,chr7,55242466,55242466,G,-AATTAAGAGAAGCAACAT,EGFR,exonic,...,NaN,0.411522633744856,2,486,1,C,1.0,True,early,True
51469,CRUK0015,CRUK0015,CRUK0015:7:55249001:-,chr7,55249001,55249001,-,CCAGCGTGT,EGFR,exonic,...,p.A767delinsASVS,0,0,102,1,C,1.0,True,early,True
52972,CRUK0004,CRUK0004,CRUK0004:7:55241708:G,chr7,55241708,55241708,G,C,EGFR,exonic,...,p.G719A,0,0,284,1,C,1.0,True,early,True
57293,CRUK0021,CRUK0021,CRUK0021:7:55259515:T,chr7,55259515,55259515,T,G,EGFR,exonic,...,p.L858R,0,0,373,1,C,1.0,True,early,True
70914,CRUK0007,CRUK0007,CRUK0007:7:55242464:A,chr7,55242464,55242464,A,-GGAATTAAGAGAAGC,EGFR,exonic,...,NaN,0,0,623,1,NaN,NaN,NaN,NA,True
73981,CRUK0297,CRUK0297,CRUK0297:7:55259515:T,chr7,55259515,55259515,T,G,EGFR,exonic,...,p.L858R,0.45,2,446,1,C,3.0,True,early,True
88259,CRUK0388,CRUK0388,CRUK0388:7:55249002:-,chr7,55249002,55249002,-,CAGCGTGGA,EGFR,exonic,...,p.A767delinsASVD,0,0,201,1,C,1.0,True,late,True
92107,CRUK0492,CRUK0492,CRUK0492:7:55242464:A,chr7,55242464,55242464,A,-GGAATTAAGAGAAGC,EGFR,exonic,...,NaN,0.143061516452074,1,699,1,S,6.0,True,subclonal,True
156585,CRUK0003,CRUK0003,CRUK0003:7:55242464:A,chr7,55242464,55242464,A,-GGAATTAAGAGAAGC,EGFR,exonic,...,NaN,0.170648464163823,1,586,1,C,1.0,True,early,True
170179,CRUK0267,CRUK0267,CRUK0267:7:55259524:T,chr7,55259524,55259524,T,A,EGFR,exonic,...,p.L861Q,0.14,1,725,1,C,1.0,True,early,True


In [66]:
clinical_df['EGFR_present']=False
for index1, row1 in mut_table[EGFR_present].iterrows():
    for index2, row2 in clinical_df.iterrows():
        if row1['patient_id'] == row2['cruk_id']:
            clinical_df.at[index2, 'EGFR_present'] = True
clinical_df.head()

for index1, row1 in EGFR_driver.iterrows():
    for index2, row2 in clinical_df.iterrows():
        if row1['patient_id'] == row2['cruk_id']:
            for i in four_genes_driver:
                if i[0] == row1['patient_id']:
                    i[2] = 1
four_genes_driver

[['CRUK0001', 1, 0, 0, 0],
 ['CRUK0002', 0, 0, 0, 0],
 ['CRUK0003', 0, 1, 0, 0],
 ['CRUK0004', 1, 1, 0, 0],
 ['CRUK0005', 1, 0, 0, 0],
 ['CRUK0006', 1, 0, 0, 0],
 ['CRUK0007', 0, 1, 0, 0],
 ['CRUK0008', 0, 0, 0, 0],
 ['CRUK0009', 1, 0, 0, 0],
 ['CRUK0010', 0, 1, 0, 0],
 ['CRUK0011', 0, 0, 0, 0],
 ['CRUK0012', 0, 1, 0, 0],
 ['CRUK0013', 0, 0, 0, 0],
 ['CRUK0014', 1, 0, 0, 0],
 ['CRUK0015', 1, 1, 0, 0],
 ['CRUK0016', 1, 0, 0, 0],
 ['CRUK0017', 1, 0, 0, 0],
 ['CRUK0018', 0, 0, 0, 0],
 ['CRUK0019', 0, 1, 0, 0],
 ['CRUK0020', 1, 0, 0, 0],
 ['CRUK0021', 1, 1, 0, 0],
 ['CRUK0022', 1, 1, 0, 0],
 ['CRUK0023', 1, 0, 0, 0],
 ['CRUK0024', 1, 0, 0, 0],
 ['CRUK0025', 1, 0, 0, 0],
 ['CRUK0026', 1, 1, 0, 0],
 ['CRUK0027', 0, 0, 0, 0],
 ['CRUK0028', 1, 0, 0, 0],
 ['CRUK0029', 1, 0, 0, 0],
 ['CRUK0030', 1, 0, 0, 0],
 ['CRUK0032', 0, 0, 0, 0],
 ['CRUK0033', 0, 0, 0, 0],
 ['CRUK0034', 0, 0, 0, 0],
 ['CRUK0035', 1, 0, 0, 0],
 ['CRUK0036', 0, 0, 0, 0],
 ['CRUK0037', 0, 0, 0, 0],
 ['CRUK0038', 0, 0, 0, 0],
 

In [15]:
CDKN2A_present = mut_table['Hugo_Symbol']=='CDKN2A'
CDKN2A_driver = mut_table[(mut_table['Hugo_Symbol'] == 'CDKN2A') & (mut_table['DriverMut'] == True)]
CDKN2A_driver

,patient_id,tumour_id,mutation_id,chr,start,stop,ref,var,Hugo_Symbol,func,...,AAChange,GL_VAF,GL_nAlt,GL_depth,ITHState,PyCloneClonal_SC,PyCloneCluster_SC,cleanCluster_SC,combTiming_SC,DriverMut
5703,CRUK0073,CRUK0073,CRUK0073:9:21971036:C,chr9,21971036,21971036,C,G,CDKN2A,exonic,...,p.D108H,0,0,231,1,C,1.0,True,early,True
15657,CRUK0269,CRUK0269,CRUK0269:9:21971111:G,chr9,21971111,21971111,G,A,CDKN2A,exonic,...,p.H83Y,0,0,307,1,C,1.0,True,early,True
41070,CRUK0588,CRUK0588,CRUK0588:9:21971120:G,chr9,21971120,21971120,G,A,CDKN2A,exonic,...,p.R80X,0,0,218,1,C,1.0,True,Unknown,True
46181,CRUK0721,CRUK0721_Tumour1,CRUK0721_Tumour1:9:21971035:T,chr9,21971035,21971035,T,A,CDKN2A,exonic,...,p.D108V,0,0,208,1,C,1.0,True,Unknown,True
70401,CRUK0178,CRUK0178,CRUK0178:9:21970912:T,chr9,21970912,21970912,T,-C,CDKN2A,exonic,...,NaN,0,0,260,1,C,1.0,True,late,True
77844,CRUK0256,CRUK0256,CRUK0256:9:21971186:G,chr9,21971186,21971186,G,A,CDKN2A,exonic,...,p.R58X,0,0,345,3,S,2.0,True,subclonal,True
80187,CRUK0240,CRUK0240,CRUK0240:9:21971036:C,chr9,21971036,21971036,C,A,CDKN2A,exonic,...,p.D108Y,0.26,1,392,1,C,1.0,True,early,True
88653,CRUK0393,CRUK0393,CRUK0393:9:21971186:G,chr9,21971186,21971186,G,A,CDKN2A,exonic,...,p.R58X,0,0,249,2,S,7.0,True,subclonal,True
92933,CRUK0456,CRUK0456,CRUK0456:9:21971053:G,chr9,21971053,21971053,G,-C,CDKN2A,exonic,...,NaN,0,0,197,1,C,1.0,True,early,True
94538,CRUK0404,CRUK0404,CRUK0404:9:21971080:G,chr9,21971080,21971080,G,C,CDKN2A,exonic,...,p.T93R,0,0,236,2,S,3.0,True,subclonal,True


In [67]:
clinical_df['CDKN2A_present']=False
for index1, row1 in mut_table[CDKN2A_present].iterrows():
    for index2, row2 in clinical_df.iterrows():
        if row1['patient_id'] == row2['cruk_id']:
            clinical_df.at[index2, 'CDKN2A_present'] = True
clinical_df.head()

for index1, row1 in CDKN2A_driver.iterrows():
    for index2, row2 in clinical_df.iterrows():
        if row1['patient_id'] == row2['cruk_id']:
            for i in four_genes_driver:
                if i[0] == row1['patient_id']:
                    i[3] = 1
four_genes_driver

[['CRUK0001', 1, 0, 0, 0],
 ['CRUK0002', 0, 0, 0, 0],
 ['CRUK0003', 0, 1, 0, 0],
 ['CRUK0004', 1, 1, 0, 0],
 ['CRUK0005', 1, 0, 0, 0],
 ['CRUK0006', 1, 0, 0, 0],
 ['CRUK0007', 0, 1, 0, 0],
 ['CRUK0008', 0, 0, 0, 0],
 ['CRUK0009', 1, 0, 0, 0],
 ['CRUK0010', 0, 1, 0, 0],
 ['CRUK0011', 0, 0, 0, 0],
 ['CRUK0012', 0, 1, 0, 0],
 ['CRUK0013', 0, 0, 0, 0],
 ['CRUK0014', 1, 0, 0, 0],
 ['CRUK0015', 1, 1, 0, 0],
 ['CRUK0016', 1, 0, 0, 0],
 ['CRUK0017', 1, 0, 0, 0],
 ['CRUK0018', 0, 0, 0, 0],
 ['CRUK0019', 0, 1, 0, 0],
 ['CRUK0020', 1, 0, 0, 0],
 ['CRUK0021', 1, 1, 0, 0],
 ['CRUK0022', 1, 1, 0, 0],
 ['CRUK0023', 1, 0, 0, 0],
 ['CRUK0024', 1, 0, 0, 0],
 ['CRUK0025', 1, 0, 0, 0],
 ['CRUK0026', 1, 1, 0, 0],
 ['CRUK0027', 0, 0, 0, 0],
 ['CRUK0028', 1, 0, 0, 0],
 ['CRUK0029', 1, 0, 0, 0],
 ['CRUK0030', 1, 0, 0, 0],
 ['CRUK0032', 0, 0, 0, 0],
 ['CRUK0033', 0, 0, 0, 0],
 ['CRUK0034', 0, 0, 0, 0],
 ['CRUK0035', 1, 0, 0, 0],
 ['CRUK0036', 0, 0, 0, 0],
 ['CRUK0037', 0, 0, 0, 0],
 ['CRUK0038', 0, 0, 0, 0],
 

In [17]:
CDKN2B_present = mut_table['Hugo_Symbol']=='CDKN2B'
CDKN2B_driver = mut_table[(mut_table['Hugo_Symbol'] == 'CDKN2B') & (mut_table['DriverMut'] == True)]
CDKN2B_driver

,patient_id,tumour_id,mutation_id,chr,start,stop,ref,var,Hugo_Symbol,func,...,AAChange,GL_VAF,GL_nAlt,GL_depth,ITHState,PyCloneClonal_SC,PyCloneCluster_SC,cleanCluster_SC,combTiming_SC,DriverMut


In [68]:
clinical_df['CDKN2B_present']=False
for index1, row1 in mut_table[CDKN2B_present].iterrows():
    for index2, row2 in clinical_df.iterrows():
        if row1['patient_id'] == row2['cruk_id']:
            clinical_df.at[index2, 'CDKN2B_present'] = True
clinical_df.head()
clinical_df.to_csv('4_genes_clinical_df.csv.gz')

for index1, row1 in CDKN2B_driver.iterrows():
    for index2, row2 in clinical_df.iterrows():
        if row1['patient_id'] == row2['cruk_id']:
            for i in four_genes_driver:
                if i[0] == row1['patient_id']:
                    i[4] = 1
four_genes_driver

[['CRUK0001', 1, 0, 0, 0],
 ['CRUK0002', 0, 0, 0, 0],
 ['CRUK0003', 0, 1, 0, 0],
 ['CRUK0004', 1, 1, 0, 0],
 ['CRUK0005', 1, 0, 0, 0],
 ['CRUK0006', 1, 0, 0, 0],
 ['CRUK0007', 0, 1, 0, 0],
 ['CRUK0008', 0, 0, 0, 0],
 ['CRUK0009', 1, 0, 0, 0],
 ['CRUK0010', 0, 1, 0, 0],
 ['CRUK0011', 0, 0, 0, 0],
 ['CRUK0012', 0, 1, 0, 0],
 ['CRUK0013', 0, 0, 0, 0],
 ['CRUK0014', 1, 0, 0, 0],
 ['CRUK0015', 1, 1, 0, 0],
 ['CRUK0016', 1, 0, 0, 0],
 ['CRUK0017', 1, 0, 0, 0],
 ['CRUK0018', 0, 0, 0, 0],
 ['CRUK0019', 0, 1, 0, 0],
 ['CRUK0020', 1, 0, 0, 0],
 ['CRUK0021', 1, 1, 0, 0],
 ['CRUK0022', 1, 1, 0, 0],
 ['CRUK0023', 1, 0, 0, 0],
 ['CRUK0024', 1, 0, 0, 0],
 ['CRUK0025', 1, 0, 0, 0],
 ['CRUK0026', 1, 1, 0, 0],
 ['CRUK0027', 0, 0, 0, 0],
 ['CRUK0028', 1, 0, 0, 0],
 ['CRUK0029', 1, 0, 0, 0],
 ['CRUK0030', 1, 0, 0, 0],
 ['CRUK0032', 0, 0, 0, 0],
 ['CRUK0033', 0, 0, 0, 0],
 ['CRUK0034', 0, 0, 0, 0],
 ['CRUK0035', 1, 0, 0, 0],
 ['CRUK0036', 0, 0, 0, 0],
 ['CRUK0037', 0, 0, 0, 0],
 ['CRUK0038', 0, 0, 0, 0],
 

In [19]:
clinical_df

,cruk_id,tumour_id_muttable_cruk,tumour_id_per_patient,age,sex,ethnicity,cigs_perday,years_smoking,packyears,smoking_status_merged,...,dfs_time_any_event,cens_lung_event,lung_event_time,Relapse_cat,Relapse_cat_new,tx100,TP53_present,EGFR_present,CDKN2A_present,CDKN2B_present
0,CRUK0034,CRUK0034,CRUK0034,68,Female,White- Irish,20.0,35,35.000,Ex-Smoker,...,1849,0,1849,NaN,No rec,True,False,False,False,False
1,CRUK0150,CRUK0150,CRUK0150,81,Male,White- British,44.5,49,109.025,Ex-Smoker,...,1362,0,1362,NaN,No rec,False,True,False,False,False
2,CRUK0159,CRUK0159,CRUK0159,60,Male,White- British,20.0,38,38.000,Smoker,...,1935,0,2224,Second primary lung,No rec,False,True,False,False,False
3,CRUK0090,CRUK0090,CRUK0090,65,Male,White- British,10.0,35,17.500,Ex-Smoker,...,1143,1,1143,Intrathoracic,Intrathoracic,True,True,False,True,False
4,CRUK0045,CRUK0045,CRUK0045,85,Male,White- British,10.0,25,12.500,Ex-Smoker,...,917,1,917,Intrathoracic,Intrathoracic,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,CRUK0872,CRUK0872,CRUK0872,66,Male,White- British,20.0,54,54.000,Ex-Smoker,...,883,1,883,Intra & Extra,Intra & Extra,False,True,False,False,False
417,CRUK0817,CRUK0817,CRUK0817,64,Male,White- British,12.0,27,16.200,Ex-Smoker,...,203,1,203,Intrathoracic,Intrathoracic,False,True,False,False,False
418,CRUK0864,CRUK0864,CRUK0864,78,Female,White- British,10.0,30,15.000,Ex-Smoker,...,1434,0,1434,NaN,No rec,False,False,False,False,False
419,CRUK0810,CRUK0810,CRUK0810,59,Female,White- British,10.0,44,22.000,Smoker,...,197,1,197,Intra & Extra,Intra & Extra,False,False,False,False,False


In [20]:
clinical_df = clinical_df.sort_values(by=['cruk_id'])
clinical_df

,cruk_id,tumour_id_muttable_cruk,tumour_id_per_patient,age,sex,ethnicity,cigs_perday,years_smoking,packyears,smoking_status_merged,...,dfs_time_any_event,cens_lung_event,lung_event_time,Relapse_cat,Relapse_cat_new,tx100,TP53_present,EGFR_present,CDKN2A_present,CDKN2B_present
88,CRUK0001,CRUK0001,CRUK0001,84,Male,White- British,20.0,30,30.00,Ex-Smoker,...,1120,0,1120,NaN,No rec,True,True,True,False,True
79,CRUK0002,CRUK0002,CRUK0002,81,Male,White- British,5.0,35,8.75,Ex-Smoker,...,1901,0,1901,NaN,No rec,True,False,False,False,False
26,CRUK0003,CRUK0003,CRUK0003,78,Female,White- British,0.0,0,0.00,Never Smoked,...,217,1,217,Intra & Extra,Intra & Extra,True,False,True,False,False
23,CRUK0004,CRUK0004,CRUK0004,69,Female,White- British,10.0,27,13.50,Ex-Smoker,...,325,1,325,Intra & Extra,Intra & Extra,True,True,True,False,False
7,CRUK0005,CRUK0005,CRUK0005,74,Female,White- British,20.0,56,56.00,Ex-Smoker,...,649,0,649,NaN,No rec,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,CRUK0861,CRUK0861,CRUK0861,65,Female,Middle eastern,10.0,45,22.50,Smoker,...,1084,0,1084,NaN,No rec,False,True,False,False,False
418,CRUK0864,CRUK0864,CRUK0864,78,Female,White- British,10.0,30,15.00,Ex-Smoker,...,1434,0,1434,NaN,No rec,False,False,False,False,False
416,CRUK0872,CRUK0872,CRUK0872,66,Male,White- British,20.0,54,54.00,Ex-Smoker,...,883,1,883,Intra & Extra,Intra & Extra,False,True,False,False,False
420,CRUK0881,CRUK0881_Tumour1,CRUK0881_Tumour1;CRUK0881_Tumour2,84,Male,White- British,20.0,70,70.00,Smoker,...,748,1,748,Intrathoracic,Intrathoracic,False,True,False,False,True


In [69]:
# Making a logistic regression algorithm to predict recurrence given the presence/absence of the genes TP53, EGFR, CDKN2A, CDKN2B

# Making the training data
X_train_4_genes_all = []
y_train = []
for index, row in clinical_df.iterrows():
    x_i = [row['TP53_present'],row['EGFR_present'],row['CDKN2A_present'],row['CDKN2B_present']]
    for j in range(len(x_i)):
        if x_i[j] == True:
            x_i[j] = 1
        elif x_i[j] == False:
            x_i[j] = 0
    X_train_4_genes_all.append(x_i)
    if row['first_dfs_any_event_rec.or.new.primary'] == 'recurrence':
        y_train.append(1)
    else:
        y_train.append(0)
print(X_train_4_genes_all)
print(y_train)
X_4_genes_all = np.array(X_train_4_genes_all)
y = np.array(y_train)

X_4_genes_driver = np.array(four_genes_driver)

[[1, 1, 0, 1], [0, 0, 0, 0], [0, 1, 0, 0], [1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 1, 0, 0], [1, 1, 0, 0], [1, 1, 0, 0], [1, 1, 0, 0], [1, 0, 0, 0], [1, 1, 0, 0], [1, 0, 0, 0], [1, 1, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [1, 0, 1, 0], [1, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 0], [0, 0, 0, 0], [1, 1, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [1, 1, 1, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 1, 1, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 1, 0, 0], [1, 0, 0, 0], [1, 0

In [70]:
# Remove the first element of every element in X_4_genes_driver
X_4_genes_driver = np.array([i[1:] for i in X_4_genes_driver])


In [74]:
X_4_genes_driver = X_4_genes_driver.astype(int)

In [75]:
X_4_genes_driver

array([[1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 0]])

In [22]:
X_train = X_4_genes_all
y_train = y
print("First five elements in X_train are:\n", X_train[:5])
print("Type of X_train:",type(X_train))
print("First five elements in y_train are:\n", y_train[:5])
print("Type of X_train:",type(y_train))
print ('The shape of X_train is: ' + str(X_train.shape))
print ('The shape of y_train is: ' + str(y_train.shape))
print ('We have m = %d training examples' % (len(y_train)))

First five elements in X_train are:
 [[1 1 0 1]
 [0 0 0 0]
 [0 1 0 0]
 [1 1 0 0]
 [1 0 0 0]]
Type of X_train: <class 'numpy.ndarray'>
First five elements in y_train are:
 [0 0 1 1 0]
Type of X_train: <class 'numpy.ndarray'>
The shape of X_train is: (421, 4)
The shape of y_train is: (421,)
We have m = 421 training examples


In [23]:
#using scikit-learn - logistic regression with split train and test groups
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_4_genes_all, y, test_size = 0.4, random_state = 0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
lr_model.score(X_test, y_test)

(252, 4) (169, 4) (252,) (169,)


0.7159763313609467

In [24]:
# cross-validation
from sklearn.model_selection import cross_val_score

lr_model = LogisticRegression()
scores = cross_val_score(lr_model, X_4_genes_all, y, cv=5)
print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

[0.65882353 0.66666667 0.66666667 0.66666667 0.6547619 ]
0.66 accuracy with a standard deviation of 0.01


In [25]:
from sklearn.model_selection import ShuffleSplit
n_samples = X_4_genes_all.shape[0]
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(lr_model, X_4_genes_all, y, cv=cv)

array([0.69291339, 0.65354331, 0.62204724, 0.68503937, 0.73228346])

In [29]:
# from ChatGPT

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

# Assuming you have a dataset with 421 examples and features, and a corresponding target variable
# Let's create a placeholder dataset for demonstration purposes
np.random.seed(42)  # For reproducibility

# Creating a logistic regression model
logreg_model = LogisticRegression()

# Performing 5-fold cross-validation
cv_scores = cross_val_score(logreg_model, X_4_genes_all, y, cv=5)

# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean CV accuracy:", np.mean(cv_scores))

Cross-validation scores: [0.65882353 0.66666667 0.66666667 0.66666667 0.6547619 ]
Mean CV accuracy: 0.6627170868347338


In [30]:
# Support vector machine (SVM) algorithm
from sklearn import svm
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_4_genes_all, y, test_size = 0.4, random_state = 0)

svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
print(svm_model)
print("Support vectors: ", svm_model.support_vectors_)
print("No. of support vectors for each class: ", svm_model.n_support_)

predictions = svm_model.predict(X_test)
acc = accuracy_score(y_test, predictions)
print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Accuracy: ",acc)

SVC()
Support vectors:  [[1. 1. 1. 0.]
 [1. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 1. 0.]
 [1. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 1. 0.]
 [1. 1. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 1. 0.]
 [0. 0. 0. 0.]
 [1. 1. 0. 0.]
 [1. 1. 0. 1.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 1. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 

In [31]:
# Neural network/Multi-layer perceptron (MLP) algorithm
from sklearn.neural_network import MLPClassifier
X_train, X_test, y_train, y_test = train_test_split(X_4_genes_all, y, test_size = 0.4, random_state = 0)

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5), random_state=1)
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)
acc = accuracy_score(y_test, predictions)
print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Accuracy: ",acc)

Predictions:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Actual:  [0 0 1 0 1 1 0 0 0 1 1 0 0 0 1 0 1 1 0 0 1 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0
 0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0
 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 0 0 0]
Accuracy:  0.7041420118343196


In [76]:
# predicting recurrence with absence/presence of 4 DRIVER genes

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_4_genes_driver, y, test_size = 0.4, random_state = 0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
lr_model.score(X_test, y_test)

(252, 4) (169, 4) (252,) (169,)


0.7159763313609467

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X_4_genes_driver, y, test_size = 0.4, random_state = 0)

svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
print(svm_model)
# print("Support vectors: ", svm_model.support_vectors_)
# print("No. of support vectors for each class: ", svm_model.n_support_)

predictions = svm_model.predict(X_test)
acc = accuracy_score(y_test, predictions)
# print("Predictions: ",predictions)
# print("Actual: ",y_test)
print("Accuracy: ",acc)

SVC()
Accuracy:  0.7100591715976331


In [78]:
# Neural network/Multi-layer perceptron (MLP) algorithm
from sklearn.neural_network import MLPClassifier
X_train, X_test, y_train, y_test = train_test_split(X_4_genes_driver, y, test_size = 0.4, random_state = 0)

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5), random_state=1)
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)
acc = accuracy_score(y_test, predictions)
# print("Predictions: ",predictions)
# print("Actual: ",y_test)
print("Accuracy: ",acc)

Accuracy:  0.7100591715976331
